### Carga y limpieza

In [ ]:
import pandas as pd

# Cargar ambas tablas
sales_final = pd.read_csv(r"C:\Users\pablo\OneDrive\Documentos\HENRY\Proyecto\Inventory\Limpios\SalesFINAL12312016_limpio.csv")
purchase_data = pd.read_csv(r"C:\Users\pablo\OneDrive\Documentos\HENRY\Proyecto\Inventory\Limpios\PurchasesFINAL_limpio.csv")


_Asignamos la columna "purchase_price" desde la base de datos "PurchasesFINAL_limpio.csv"_

In [ ]:
# Crear diccionario de precios promedio por Brand
price_map = purchase_data.groupby('brand')['purchase_price'].mean().to_dict()

# Asignar a SalesFinal sin merge
sales_final['purchase_price'] = sales_final['brand'].map(price_map)

In [ ]:
# Selección de variables relevantes
features = sales_final[['sales_quantity', 'sales_dollars', 'purchase_price', 'volume', 'store']]

Utilizando 'sales_quantity' como variable objetivo relizamos los cortes para hacer las clasificaciones

In [ ]:
# Definimos las categorías de rotación de stock
def clasificar_rotacion(row):
    if row['inventory_level'] > 3 * row['sales_quantity']:
        return 'Sobrestock'
    elif row['sales_quantity'] < 10:
        return 'Baja'
    elif row['sales_quantity'] < 100:
        return 'Media'
    else:
        return 'Alta'

sales_final['rotacion_categoria'] = sales_final.apply(clasificar_rotacion, axis=1)

NameError: name 'sales_final' is not defined

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Codificar variables categóricas
features_encoded = pd.get_dummies(features)

# Codificar la variable objetivo
le = LabelEncoder()
target = le.fit_transform(df['rotacion_categoria'])

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier()
model.fit(X_train, y_train)